This uses the open AI gym baseline with some slight modifications. Mostly copied code from here
https://github.com/openai/baselines/tree/master/baselines/ppo1. Looks like the version I got from pip and the version currently on master don't quite sync up. (For example there is no tf_util.save_state fcn so we save manually

In [1]:
from baselines.common.cmd_util import make_mujoco_env, mujoco_arg_parser
from baselines.common import tf_util as U
import tensorflow as tf
from baselines import logger
import os
from baselines.ppo1 import mlp_policy, pposgd_simple


def train(env_id, num_timesteps, seed=0):

    
    U.make_session(num_cpu=1).__enter__()
    
    def policy_fn(name, ob_space, ac_space):
        return mlp_policy.MlpPolicy(name=name, ob_space=ob_space, ac_space=ac_space, hid_size=64, num_hid_layers=2)

    env = make_mujoco_env(env_id, seed)
    pi = pposgd_simple.learn(env, policy_fn,
            max_timesteps=num_timesteps,
            timesteps_per_actorbatch=2048,
            clip_param=0.2, entcoeff=0.0,
            optim_epochs=10, optim_stepsize=3e-4, optim_batchsize=64,
            gamma=0.99, lam=0.95, schedule='linear',
        )
    env.close()
   
    return pi



   

/Users/sgillen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
logger.configure()
train("Walker2d-v2", num_timesteps=5000000, seed=0)
U.save_state("./saved/5mil_flat")

In [4]:
#def loaded_policy_fn(name, ob_space, ac_space):
#    return mlp_policy.MlpPolicy(name=name, ob_space=ob_space, ac_space=ac_space, hid_size=64, num_hid_layers=2)

#loader = tf.train.Saver()
#session = loader.restore(tf.get_default_session(), 'saved/test.save')

logger.configure()

pi = train('Walker2d-v2', num_timesteps=1, seed=0)
U.load_state('./test.save')
#os.makedirs(os.path.dirname("./saved"), exist_ok=True)
#saver = tf.train.Saver()
#saver.save(tf.get_default_session(), "./saved/test.save")
    
env = make_mujoco_env('Walker2d-v2',seed=0)
ob = env.reset()     

while True:
    action = pi.act(stochastic=False, ob=ob)[0]
    ob, _, done, _ =  env.step(action)
    env.render()
    if done:
        ob = env.reset()

Logging to /var/folders/qq/gpxz4l6s1tndfdhysbz8bdym0000gn/T/openai-2018-08-27-17-58-49-567478
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
********** Iteration 0 ************
Optimizing...
     pol_surr |    pol_entpen |       vf_loss |            kl |           ent
     -0.00675 |       0.00000 |       9.48014 |       0.00147 |       8.51539
     -0.02158 |       0.00000 |       8.28080 |       0.00637 |       8.51047
     -0.02782 |       0.00000 |       7.47112 |       0.00898 |       8.50481
     -0.03148 |       0.00000 |       6.89381 |       0.01096 |       8.49936
     -0.03413 |       0.00000 |       6.47413 |       0.01110 |       8.49700
     -0.03660 |       0.00000 |       6.17714 |       0.01175 |       8.49347
     -0.03829 |       0.00000 |       5.96013 |       0.01336 |       8.48884
     -0.03964 |       0.00000 |   

ValueError: The passed save_path is not a valid checkpoint: ./test.save

In [4]:
env = make_mujoco_env("Walker2d-v2", 0)
ob_space = env.observation_space
ac_space = env.action_space